# Week 7 Exercises


## Exercise 1: Vectorized  derivatives for Neural Nets - single data point

Let $x$ be a data point ($1 \times d$) vector and $W_1$ a weight matrix of shape $d \times h$ used in the neural net 
$$
\textrm{nn}(x) = (xW_1 + b_1)W_2 + b_2
$$
Let $e$ be the error function considered

Furthermore, let
$$
s = x W_1, \quad v = \frac{\partial e}{\partial s} = \left[\frac{\partial e}{\partial s[a]}, a=1,\dots, h\right]
$$ 
both $1 \times h$ vectors

We need to check that 

$$
\frac{\partial e}{\partial W_1} = x^\intercal v
$$
as claimed in the lecture. However we do not want to compute $\frac{\partial s}{\partial W_1}$ as this gives a $h \times (d \times h)$ tensor or  a $h \times (d * h)$ matrix (flattened $W_1$)

We will simply check the formula by deriving
$$
\frac{\partial e}{\partial W_1[i, j]}
$$
a derivative of just one entry in $W_1$ and see that it fits (considering $e$ relative to just one weight in $W_1$, then it is a function from 1 variable to 1 number and similarly $\frac{\partial s}{\partial W_1[i,j]}$  considers a function from 1 number to $h$ numbers).



**TASK:**
- Step 1: Derive $\frac{\partial s}{\partial W_1[i,j]} = \left[\frac{\partial s[a]}{\partial W_1[i, j]}, a=1,\dots, h\right]$  - it is a $h \times 1$ vector. **Hint:** it contains mostly zeros except at one important place.
- Step 2: Use Chain Rule to show that $\frac{\partial e}{\partial W_1[i, j]} = x_i v_j$ and conclude the formula fits


## Exercise 2:  Vectorized  derivatives for Neural Nets - batch of data
With the solution from Exercise 1 in place, we can now try and optimize for the case where we replace the single data point $x$ with a mini-batch of data $X$. We use the same neural net function as in Exercise 1.

Let $X$ be a mini-batch of data points ($n \times d$) matrix and $W_1$ a weight matrix of shape $d \times h$
To evaluate all elements in the minibatch we get

$$
\textrm{nn}(X) = (XW_1 + b_1)W_2 + b_2
$$

which outputs an $n \times 1$ vector instead of just one number.

The error function considered is the sum of errors over the data in the batch  
$$
\sum_{a=1}^n e(y_a, \textrm{nn}(x_a))
$$ 

where $e$ is some error function as above, and $x_a$ is the $a'th$ row in X (as a $1 \times d$ vector) and $y_a$ is the target value for $x_a$.

Furthermore, let

$$
S = X W_1, \quad V = \frac{\partial e}{\partial S} 
$$ 

both $n \times h$ matrices, having a row for every input point $x \in X$. This is the same as above applied for the $n$ points instead of one and placed in the rows of matrices $S$ and $V$.

We know that if $X$ (and hence $S, V$) only has one row then the result is given above.
To get derivatives of the sum we must sum the derivative and hence
$$
 \sum_{a=1}^ n \frac{\partial e(y_a, \textrm{nn}(x_a))}{\partial W} = \sum_{a=1}^n x_a^\intercal v_a
$$
where $x_a$ is the $a$'th row of $X$ and $v_a$ is the $a$'th row of V.


**Task:** 
Show how to rewrite this sum into one matrix product and argue correctness.


## Exercise 3: Convolution and Pooling
In this exercise we will see if we can get a better grasp of convolution and pooling.
Your job is to implement basic convolution and pooling.

For this we need the python package pillow  which you need to install

- The Convolution operator that takes a $d \times d$ weight matrix and 1 channel image, and applies the convolution with the weight vector with the image.
- The max pooling operator takes an input image and a max pooling size and returns the pooled output.
    For simplicity we only consider 2 x 2 max pooling
- Test your convolution implementation with the $3 \times 3$ matrix with -1 everywhere exept the middle and 8 in the middle is a classic edge detector pattern.
- Test your pooling implementation by applying a $2 \times 2$ max pooling to the output of the convolution.

We assume that we pad the input to ensure the output of the convolution is the same width and height as the input image.

To compare your implementation we have supplied code that applies the convolution and the pooling operator from the neural net package in pytorch.





In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image #(pacakge name is pillow - i.e. pip3 install pillow )
import os
filename = 'lena512_gray.bmp'

img = Image.open('lena512_gray.bmp')
lena = np.array(img)
print('image shape', lena.shape)
plt.imshow(lena, cmap='gray')
plt.show()

def conv2d(img, w):
    """ Return the result of applying the convolution defined by w to img - 
    for simplicity assume that w is square"""
    w_dim = w.shape[0]
    pad = w_dim - 2
    padded_img = np.pad(img, [pad, pad], 'constant', constant_values=0)    
    out = np.zeros(img.shape)
    ### YOUR CODE HERE
    ### END CODE
    return out

def max_pool2d(img):
    """ Return the result of applying the 2 x 2 max pooling operator to mig (halve the width and height of image)"""
    out = np.zeros((int(img.shape[0]/2), int(img.shape[1]/2)))
    ### YOUR CODE HERE
    ### END CODE
    return out

conv_filter = np.array([[-1., -1., -1.], [-1., 8, -1.], [-1., -1., -1.]])
convoluted_lena = conv2d(lena, conv_filter)
pooled_lena = max_pool2d(convoluted_lena)
fig, axes = plt.subplots(1, 2, figsize=(20, 16))
axes[0].imshow(convoluted_lena, cmap='gray')
axes[1].imshow(pooled_lena, cmap='gray')
plt.show()

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
torch_lena = torch.from_numpy(lena).view(1, 1, lena.shape[0], lena.shape[1]).double()
print('image shape', torch_lena.shape)
tv = torch.tensor([[-1., -1., -1.], [-1., 8, -1.], [-1., -1., -1.]])
tv = tv.view(1, 1, 3, 3).double()
torch_convoluted_lena = F.conv2d(torch_lena, tv, torch.tensor([0.], dtype=torch.double), 1, 1, 1, 1)
print('convoluted_lena shape', torch_convoluted_lena.shape)
print('conv diff norm', np.linalg.norm(torch_convoluted_lena.numpy().squeeze() - convoluted_lena))
torch_pooled_lena = F.max_pool2d(torch_convoluted_lena, kernel_size=(2, 2))
print('pool diff norm', np.linalg.norm(torch_pooled_lena.numpy().squeeze() - pooled_lena))
fig, axes = plt.subplots(1, 2, figsize=(20, 16))
axes[0].imshow(torch_convoluted_lena.numpy().squeeze(), cmap='gray')
axes[1].imshow(torch_pooled_lena.numpy().squeeze(), cmap='gray')
plt.show()

# Exercise 4: How Many Parameters Does the VGG net use
The VGG net is a well known neural net architecture for image recognition. 

**How many parameters does it use? Write down a formula and compute the number!**

It has the following architecture [https://pytorch.org/docs/0.4.0/_modules/torchvision/models/vgg.html]

It takes as input an image of size 224 x 224 x 3.

First it computes 64, 3 x 3 x 3 convolutions i.e. makes 64 channels. Then it makes 64, 3 x 3 x 64 convolutionas on the output of the previous convolution i.e. it makes 64 new channels. This is followd by  a $2 \times 2$ max pooling operation.

The pattern continues as described below: A number is the number of new convolutions made on previous input and 'M' means max pooling.

64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'

All convolutions are 3x3 using padding of 1 of zeros around the input so the convolution output size has height and width the same as the input.
All poolings are $2 \times 2$ non-overlapping (stride=2)

After the fully convolutional layers it used standard fully connected layers as
- 512 * 7 * 7, 4096 (why 512 * 7 * 7?)
- 4096, 4096,
- 4096, 1000
        



## Exercise 5:  Maximal Margin By Hand 

- Assume you are given two points $x_1=(1, 1)$ with class -1 and $x_2 = (11, 11)$ with class 1. What is the maximum separating hyperplane. E.g. what are vectors w and b that maximally separates these points? What are the support vectors? What is the margin? Can you find the parameters w, b the SVM wil find without trying to solve the SVM problem directly but instead thinking a little. Try visualizing the data set.
- If we have three points in class -1 (-10, -10), (-5, 2), (1, 1) and four points in class 1 (20, 23), (15, 17), (12, 10), (10, 12). What is the maximal separating hyperplane (w,b)? Try visualizing the data set before you work to hard.
What are the support vectors? What is the margin? You can run the code below the next exercise to get the answers from actually running the python sklearn SVM implementation on the data.
- Write down the exact form of the SVM problem we need to solve if the input data is defined as the 7 points above.
The convex quadratic program was defined as 

$\min_{w, b} \frac{1}{2}{||w||^2}$

s.t. $y_i(w^\intercal x_i + b) \geq 1$

You job is thus to write down the constraints.


.

## Exercise 6: Regularized SVM

Consider a regularized SVM wrong side of the margin penalty C.

$\min_{w, b, \xi} \frac{1}{2}{||w||^2} + C \sum_{i=1}^n \xi_i$

s.t. $y_i(w^\intercal x_i + b) \geq 1 - \xi_i$

and $\xi_i \geq 0$

Assume you are given three points $x_1=(1, 1)$ with class -1 and $x_2=(3, 3), x_3 = (11, 11)$ with class 1.
- Write down the exact form of the SVM problem we need to solve if the input data $D=\{x_1, x_2, x_3\}$.
  Thus, your job is thus to write down the constraints.

- What is the best cost you can get when using the hyperplane $w = [ 0.1, 0.1]$ and  $b: -1.2$. i.e. how can you pick $\xi_1, \xi_2, \xi_3$ such that the constraints are satisfied while minimizing the cost with w and b  fixed.

- Is there a general way given, $w$ and $b$ to compute the best $\xi_i$ 
- Can you find a $w, b$ with a smaller cost for $C=1$
  If you like you can use the svm code below to experiment.
  


Also be sure to see how to apply an SVM implementation on data using Sklearn.


In [ ]:
# code for running the example above
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm

def plot_hyperplane(w, *args, **kwargs): 
    if w[1]==0 and w[2]==0:
        raise ValueError('Invalid hyperplane')
    xmin, xmax, ymin, ymax = plt.axis()
    
    if w[2]==0:
        # Vertical line
        x = np.array((1/w[1], 1/w[1]))
        y = np.array((ymin, ymax))
    else:
        x = np.array((xmin, xmax))
        y = (-w[0]-w[1]*x)/w[2]       
    # plot the line
    plt.plot(x, y, *args, **kwargs)

def run_svm(X, Y, kernel='linear', **kwargs):
    # fit the model
    clf = svm.SVC(kernel=kernel, **kwargs)
    clf.fit(X, Y)

    # get the separating hyperplane
    
    print('Hyperplane found w: {0} b: {1}'.format(clf.coef_[0],clf.intercept_[0]))
    margin = 1.0/np.linalg.norm(clf.coef_[0])
    print('Margin 1/||w||: {0}'.format(margin))
    hyp = np.r_[clf.intercept_,clf.coef_[0]]
    
    plt.figure(figsize=(10, 8))
    plt.scatter(X[:, 0], X[:, 1], c=Y,cmap=plt.cm.Paired)
    print('Support Vectors:')    
    print(clf.support_vectors_)
    
    plt.scatter(clf.support_vectors_[:, 0], clf.support_vectors_[:, 1],s=80, facecolors='none')
    plt.xlim(X.min()-1,X.max()+1)
    plt.ylim(X.min()-1,X.max()+1)
    plot_hyperplane(hyp,'k-')
    
    print('what is hyp',hyp)
    tmp = hyp[0]
    hyp[0] = tmp + 1
    plot_hyperplane(hyp,'k--')
    hyp[0] = tmp -1
    plot_hyperplane(hyp,'k--')
    hyp[0] = tmp
    
    #plt.axis('tight')
    plt.show()

X = np.array([[1, 1], [11, 11]])
Y = np.array([0, 1])
print('First Exercise From Above')
run_svm(X, Y, C=1)
plt.show()

print('Second Exercise From Above')
X = np.array([(-10, -10), (-5, 2), (1, 1),  (20, 23), (15, 17), (12, 10), (10, 12)])
Y = np.array([0, 0, 0, 1, 1, 1, 1])
run_svm(X, Y)
plt.show()

# print('Larger Separable Data set')
# n = 50
# X = 2*np.r_[2*np.random.randn(n, 2) - [4, 4], 2*np.random.randn(n, 2) + [5, 5]]
# Y = [-1] * n + [1] * n
# run_svm(X, Y)
# plt.show()
